### QA Type 15 : Basic Deduction

Task 15 tests basic deduction via inheritance of properties, e.g. “Sheep are afraid of wolves. Gertrude is a sheep. What is Gertrude afraid of?”. 

For training and testing we have used the 10k dataset, containing 10,000 training examples.
An example QA set is as below -:

1. Mice are afraid of wolves.
2. Gertrude is a mouse.
3. Cats are afraid of sheep.
4. Winona is a mouse.
5. Sheep are afraid of wolves.
6. Wolves are afraid of cats.
7. Emily is a mouse.
8. Jessica is a wolf.
9. What is gertrude afraid of?	wolf	

In [1]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers import Activation, Dense, Merge, Permute, Dropout
from keras.layers import LSTM
from keras.utils.data_utils import get_file
from keras.preprocessing.sequence import pad_sequences
from functools import reduce
import tarfile
import numpy as np
import re

Using TensorFlow backend.


### Story processing

Each story is processed to reduce sentences to tokens and combine all facts and queries relevant to the story. 

In [2]:
def tokenize(sent):
    '''Return the tokens of a sentence including punctuation.
    >>> tokenize('Bob dropped the apple. Where is the apple?')
    ['Bob', 'dropped', 'the', 'apple', '.', 'Where', 'is', 'the', 'apple', '?']
    '''
    return [x.strip() for x in re.split('(\W+)?', sent) if x.strip()]


In [3]:
def parse_stories(lines, only_supporting=False):
    '''Parse stories provided in the bAbi tasks format
    If only_supporting is true, only the sentences that support the answer are kept.
    '''
    data = []
    story = []
    for line in lines:
        line = line.decode('utf-8').strip()
        nid, line = line.split(' ', 1)
        nid = int(nid)
        if nid == 1:
            story = []
        if '\t' in line:
            q, a, supporting = line.split('\t')
            q = tokenize(q)
            substory = None
            if only_supporting:
                # Only select the related substory
                supporting = map(int, supporting.split())
                substory = [story[i - 1] for i in supporting]
            else:
                # Provide all the substories
                substory = [x for x in story if x]
            data.append((substory, q, a))
            story.append('')
        else:
            sent = tokenize(line)
            story.append(sent)
    return data

In [4]:

def get_stories(f, only_supporting=False, max_length=None):
    '''Given a file name, read the file, retrieve the stories, and then convert the sentences into a single story.
    If max_length is supplied, any stories longer than max_length tokens will be discarded.
    '''
    data = parse_stories(f.readlines(), only_supporting=only_supporting)
    flatten = lambda data: reduce(lambda x, y: x + y, data)
    data = [(flatten(story), q, answer) for story, q, answer in data if not max_length or len(flatten(story)) < max_length]
    return data


In [5]:
def vectorize_stories(data, word_idx, story_maxlen, query_maxlen):
    X = []
    Xq = []
    Y = []
    for story, query, answer in data:
        x = [word_idx[w] for w in story]
        xq = [word_idx[w] for w in query]
        y = np.zeros(len(word_idx) + 1)  # let's not forget that index 0 is reserved
        y[word_idx[answer]] = 1
        X.append(x)
        Xq.append(xq)
        Y.append(y)
    return (pad_sequences(X, maxlen=story_maxlen),
            pad_sequences(Xq, maxlen=query_maxlen), np.array(Y))

In [6]:
try:
    path = get_file('babi-tasks-v1-2.tar.gz', origin='https://s3.amazonaws.com/text-datasets/babi_tasks_1-20_v1-2.tar.gz')
except:
    print('Error downloading dataset, please download it manually:\n'
          '$ wget http://www.thespermwhale.com/jaseweston/babi/tasks_1-20_v1-2.tar.gz\n'
          '$ mv tasks_1-20_v1-2.tar.gz ~/.keras/datasets/babi-tasks-v1-2.tar.gz')
    raise
tar = tarfile.open(path)

### Training Set

Here the selection of 'Question' type (single supporting fact, three supporting facts etc.) is made for generating the training and test set. 

In [7]:
challenges = {
    'single_supporting_fact_10k': 'tasks_1-20_v1-2/en-10k/qa1_single-supporting-fact_{}.txt',
    'two_supporting_facts_10k': 'tasks_1-20_v1-2/en-10k/qa2_two-supporting-facts_{}.txt',
    'three_supporting_facts_10k': 'tasks_1-20_v1-2/en-10k/qa2_three-supporting-facts_{}.txt',
    'two-arg-relations_10k' : 'tasks_1-20_v1-2/en-10k/qa4_two-arg-relations_{}.txt',
    'three-arg-relations_10k' : 'tasks_1-20_v1-2/en-10k/qa5_three-arg-relations_{}.txt',
    'yes-no-questions_10k' :'tasks_1-20_v1-2/en-10k/qa6_yes-no-questions_{}.txt',
    'counting_10k':'tasks_1-20_v1-2/en-10k/qa7_counting_{}.txt',
    'lists-sets_10k' : 'tasks_1-20_v1-2/en-10k/qa8_lists-sets_{}.txt',
    'simple-negation_10k':'tasks_1-20_v1-2/en-10k/qa9_simple-negation_{}.txt',
    'indefinite-knowledge_10k' : 'tasks_1-20_v1-2/en-10k/qa10_indefinite-knowledge_{}.txt',
    'basic-coreference_10k':'tasks_1-20_v1-2/en-10k/qa11_basic-coreference_{}.txt',
    'conjunction_10k':'tasks_1-20_v1-2/en-10k/qa12_conjunction_{}.txt',
    'compound-coreference_10k':'tasks_1-20_v1-2/en-10k/qa13_compound-coreference_{}.txt',
    'time-reasoning_10k':'tasks_1-20_v1-2/en-10k/qa14_time-reasoning_{}.txt',
    'basic-deduction_10k' : 'tasks_1-20_v1-2/en-10k/qa15_basic-deduction_{}.txt',
    'basic-induction_10k':'tasks_1-20_v1-2/en-10k/qa16_basic-induction_{}.txt',
    'positional-reasoning_10k':'tasks_1-20_v1-2/en-10k/qa17_positional-reasoning_{}.txt',
    'size-reasoning_10k':'tasks_1-20_v1-2/en-10k/qa18_size-reasoning_{}.txt',
    'path-finding_10k':'tasks_1-20_v1-2/en-10k/qa19_path-finding_{}.txt',
    'agents-motivations_10k':'tasks_1-20_v1-2/en-10k/qa20_agents-motivations_{}.txt'
    
}
challenge_type = 'basic-deduction_10k'
challenge = challenges[challenge_type]

print('Extracting stories for the challenge:', challenge_type)
train_stories = get_stories(tar.extractfile(challenge.format('train')))
test_stories = get_stories(tar.extractfile(challenge.format('test')))


Extracting stories for the challenge: basic-deduction_10k


In [8]:
vocab = sorted(reduce(lambda x, y: x | y, (set(story + q + [answer]) for story, q, answer in train_stories + test_stories)))
# Reserve 0 for masking via pad_sequences
vocab_size = len(vocab) + 1
story_maxlen = max(map(len, (x for x, _, _ in train_stories + test_stories)))
query_maxlen = max(map(len, (x for _, x, _ in train_stories + test_stories)))

print('Vocab size:', vocab_size, 'unique words')
print('Story max length:', story_maxlen, 'words')
print('Query max length:', query_maxlen, 'words')
print('Number of training stories:', len(train_stories))
print('Number of test stories:', len(test_stories))
print('\n')
print('Here\'s what a "story" tuple looks like (input, query, answer):')
print(train_stories[0])

Vocab size: 28 unique words
Story max length: 44 words
Query max length: 6 words
Number of training stories: 10000
Number of test stories: 1000


Here's what a "story" tuple looks like (input, query, answer):
([u'Mice', u'are', u'afraid', u'of', u'wolves', u'.', u'Gertrude', u'is', u'a', u'mouse', u'.', u'Cats', u'are', u'afraid', u'of', u'sheep', u'.', u'Winona', u'is', u'a', u'mouse', u'.', u'Sheep', u'are', u'afraid', u'of', u'wolves', u'.', u'Wolves', u'are', u'afraid', u'of', u'cats', u'.', u'Emily', u'is', u'a', u'mouse', u'.', u'Jessica', u'is', u'a', u'wolf', u'.'], [u'What', u'is', u'gertrude', u'afraid', u'of', u'?'], u'wolf')


### Vectorizing the word sequences

Here we convert the word tokens to word vectors. We know the total number of unique words in the corpus, that is the size of the vocabulary. Thus, to each word in the corpus we assign a number. We also know the maximum length of a story, story_maxlen (story includes facts, query and answer). Thus, for each story we can construct a word vector of length story_maxlen. For eg. suppose, story_maxlen is 68 and the current story length is 24, then we can have a word vector with the first 34 indices as 0 and the next 24 indices with vocabulary index values of words in the current story

In [9]:
word_idx = dict((c, i + 1) for i, c in enumerate(vocab))
inputs_train, queries_train, answers_train = vectorize_stories(train_stories, word_idx, story_maxlen, query_maxlen)
inputs_test, queries_test, answers_test = vectorize_stories(test_stories, word_idx, story_maxlen, query_maxlen)


print('inputs: integer tensor of shape (samples, max_length)')
print('inputs_train shape:', inputs_train.shape)
print('inputs_test shape:', inputs_test.shape)
print('\n')
print('queries: integer tensor of shape (samples, max_length)')
print('queries_train shape:', queries_train.shape)
print('queries_test shape:', queries_test.shape)
print('\n')
print('answers: binary (1 or 0) tensor of shape (samples, vocab_size)')
print('answers_train shape:', answers_train.shape)
print('answers_test shape:', answers_test.shape)
print('\n')


inputs: integer tensor of shape (samples, max_length)
inputs_train shape: (10000, 44)
inputs_test shape: (1000, 44)


queries: integer tensor of shape (samples, max_length)
queries_train shape: (10000, 6)
queries_test shape: (1000, 6)


answers: binary (1 or 0) tensor of shape (samples, vocab_size)
answers_train shape: (10000, 28)
answers_test shape: (1000, 28)




### End-to-End Memory Networks

The model employed is the End-to-End Memory Networks described in Sukhbaatar et. al. (2015). The architecture is a modified form of Memory Network (Weston, 2014) with the training being weakly supervised.Weston's Memory Network was not easy to train via backpropagation and required supervision at each layer.


Briefly, the model takes a discrete set of inputs x1, x2, ...xn that are to be stored in memory, a query q, and an answer a. The model writes all x to the memory up to a fixed buffer size, and then finds a continuous representation for x and q. The continuous representation is then processed via multiple hops to output a. 


In [10]:
# embed the input sequence into a sequence of vectors
# output: (samples, story_maxlen, embedding_dim)
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_size,
                              output_dim=64,
                              input_length=story_maxlen))
input_encoder_m.add(Dropout(0.3))


# embed the question into a sequence of vectors
# output: (samples, query_maxlen, embedding_dim)
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=vocab_size,
                               output_dim=64,
                               input_length=query_maxlen))
question_encoder.add(Dropout(0.3))

# compute a 'match' between input sequence elements (which are vectors)
# and the question vector sequence

# output: (samples, story_maxlen, query_maxlen)
match = Sequential()
match.add(Merge([input_encoder_m, question_encoder],
                mode='dot',
                dot_axes=[2, 2]))
match.add(Activation('softmax'))

# embed the input into a single vector with size = story_maxlen:
# output: (samples, story_maxlen, query_maxlen)
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_size,
                              output_dim=query_maxlen,
                              input_length=story_maxlen))
input_encoder_c.add(Dropout(0.3))


# sum the match vector with the input vector:
# output: (samples, story_maxlen, query_maxlen)
response = Sequential()
response.add(Merge([match, input_encoder_c], mode='sum'))

response.add(Permute((2, 1)))  # output: (samples, query_maxlen, story_maxlen)

# concatenate the match vector with the question vector,
# and do logistic regression on top
answer = Sequential()
answer.add(Merge([response, question_encoder], mode='concat', concat_axis=-1))


# the original paper uses a matrix multiplication for this reduction step.
# we choose to use a RNN instead.
answer.add(LSTM(32))
# one regularization layer -- more would probably be needed.
answer.add(Dropout(0.3))
answer.add(Dense(vocab_size))
# we output a probability distribution over the vocabulary
answer.add(Activation('softmax'))



In [11]:
answer.compile(optimizer='rmsprop', loss='categorical_crossentropy',
               metrics=['accuracy'])
# Note: you could use a Graph model to avoid repeat the input twice
answer.fit([inputs_train, queries_train, inputs_train], answers_train,
           batch_size=32,
           nb_epoch=60,
           validation_data=([inputs_test, queries_test, inputs_test], answers_test))

Train on 10000 samples, validate on 1000 samples
Epoch 1/60
10000/10000 [==============================] - 15s - loss: 1.6065 - acc: 0.2500 - val_loss: 1.3983 - val_acc: 0.2410
Epoch 2/60
10000/10000 [==============================] - 12s - loss: 1.3361 - acc: 0.3520 - val_loss: 1.2915 - val_acc: 0.3620
Epoch 3/60
10000/10000 [==============================] - 15s - loss: 1.2709 - acc: 0.4179 - val_loss: 1.1893 - val_acc: 0.4730
Epoch 4/60
10000/10000 [==============================] - 12s - loss: 1.1932 - acc: 0.4809 - val_loss: 1.1241 - val_acc: 0.5030
Epoch 5/60
10000/10000 [==============================] - 15s - loss: 1.1513 - acc: 0.5026 - val_loss: 1.0892 - val_acc: 0.5270
Epoch 6/60
10000/10000 [==============================] - 10s - loss: 1.1233 - acc: 0.5121 - val_loss: 1.1332 - val_acc: 0.5030
Epoch 7/60
10000/10000 [==============================] - 14s - loss: 1.1000 - acc: 0.5228 - val_loss: 1.0243 - val_acc: 0.5340
Epoch 8/60
10000/10000 [==============================]

### Making predictions based on the fitted model

In [12]:
predictions=answer.predict_classes([inputs_test, queries_test, inputs_test], batch_size=32)

1000/1000 [==============================] - 0s     


### Example story and model prediction

In [13]:
def pred_evaluation(ind):
    
    print ("The story is -: ")
    facts = test_stories[ind]
    print(' '.join(facts[0]))
    print('\n')
    
    print ("The query is -:")
    print(' '.join(facts[1]))

    print ("\nThe predicted answer is -:")
    ans = predictions[ind]
    for key, value in (word_idx).items():
        if value == ans:
            print(key)
            print('\n')
            


In [14]:
pred_evaluation(1)


The story is -: 
Wolves are afraid of mice . Sheep are afraid of mice . Winona is a sheep . Mice are afraid of cats . Cats are afraid of wolves . Jessica is a mouse . Emily is a cat . Gertrude is a wolf .


The query is -:
What is winona afraid of ?

The predicted answer is -:
mouse




In [15]:
pred_evaluation(20)


The story is -: 
Cats are afraid of wolves . Mice are afraid of sheep . Sheep are afraid of mice . Emily is a sheep . Wolves are afraid of sheep . Gertrude is a sheep . Winona is a wolf . Jessica is a mouse .


The query is -:
What is gertrude afraid of ?

The predicted answer is -:
mouse




In [16]:
pred_evaluation(540)

The story is -: 
Mice are afraid of wolves . Sheep are afraid of mice . Wolves are afraid of sheep . Gertrude is a mouse . Cats are afraid of sheep . Winona is a cat . Jessica is a wolf . Emily is a mouse .


The query is -:
What is gertrude afraid of ?

The predicted answer is -:
sheep


